## Runs DE on different clusters. Also runs DE within each cluster for each condition.
Does this over each donor and combined

In [1]:
# Input info
se_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/SE.rds"
sample_names = "preB,postB"
outdir = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/peaks_donors_DE_minPct_00"

# Parameters
assay="ATAC"
minPct = 0.00
logfcthresh = 0.25
top_de = 3
p_thresh=0.1
test.use="LR"
latent.vars = "peak_region_fragments"

# Parallel processing
cores = 4

# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"



In [2]:
source(utils_script)
outdir = outdir
dir.create(outdir, showWarnings = FALSE)

btwnConds_inClust_sepDonrs.outdir <- file.path(outdir, "btwnConds_inClust_sepDonrs")
dir.create(btwnConds_inClust_sepDonrs.outdir)

btwnConds_inClust.outdir <- file.path(outdir, "btwnConds_inClust")
dir.create(btwnConds_inClust.outdir, showWarnings = FALSE)

Attaching SeuratObject

Warning message in dir.create(btwnConds_inClust_sepDonrs.outdir):
“'/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/peaks_donors_DE_minPct_00/btwnConds_inClust_sepDonrs' already exists”


In [3]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
sample_names

[1] "preB"  "postB"

In [4]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")

library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

Lo

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [5]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
DefaultAssay(se) <- assay
se

# Stimulus specific response in each cluster 

## Each donor separately

In [ ]:
se$condition.name <- as.factor(apply(se[[]][ , c("orig.ident", "seurat_clusters", "donor") ] , 1 , paste , collapse = "_" ))
se$condition.clust <- as.factor(apply(se[[]][ , c("orig.ident", "seurat_clusters") ] , 1 , paste , collapse = "_" ))

In [ ]:
# get clust and donor information
source(utils_script)
clust.ids <- levels(se$seurat_clusters)
don.ids <- levels(se$donor)
#cols <- c("seurat_clusters", "donor")
curr.ids <- levels(as.factor(apply(se[[]][ , c( "seurat_clusters", "donor") ] , 1 , paste , collapse = "_" )))
# Setup Idents to include the condition, clusters and donor
Idents(se) <- as.factor(apply(se[[]][ , c("orig.ident", "seurat_clusters", "donor") ] , 1 , paste , collapse = "_" ))
se$curr.id = Idents(se)

# setup output
# all.pvals is gene-by-comparison df of the pvalue results
all.pvals <- data.frame(matrix(nrow=dim(se)[1],ncol=0),
                          row.names=row.names(se))
all.names.sig <- c()

# Loop through clusters and donors and compare between conditions
for (cl in clust.ids){
    if (cl == "None"){
        next
    }
    for (d in don.ids){
        if (d == "None"){
            print('Skipping None')
            next
        }
        c <- paste0(cl, "_", d)
        cl.names.sig = c()
        
        try({
            print("c")
            print(c)
            id1 <- paste(sample_names[[1]], c, sep="_")
            id2 <- paste(sample_names[[2]], c, sep="_")         
            curr.name <- c
            curr.name <- paste(c, sample_names[[1]], sample_names[[2]], sep='_')
            curr.clust.donor.outdir <- file.path(btwnConds_inClust_sepDonrs.outdir,
                                                 paste0("clust_", cl, "donor_", d))
            dir.create(curr.clust.donor.outdir)

            curr.sig <- find.markers.and.plot(se, id1, id2, curr.clust.donor.outdir, 
                                              curr.name=curr.name, 
                                              latent.var=latent.vars, test.use=test.use,
                                              min.pct=minPct, p.thresh=p_thresh)
            if (length(names(curr.sig)) > 0){
                print("adding sig vals to all pvals")
                names.sig <- rownames(curr.sig)
                all.names.sig <- c(all.names.sig, head(names.sig, top_de))
                all.pvals[names.sig,curr.name] <- curr.sig$p_val_adj_BH
            }else{
                print("no sig variables found")
            }
        })
    }
}

## Combined DE results

## i. p-values ordered by average -log p-value if they were significant (p<0.1 here)

In [ ]:
sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
dot <- DotPlot(se, scale=FALSE,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=T) + RotatedAxis()
dot

de.plots(se, names(sig.all.ordered), btwnConds_inClust_sepDonrs.outdir, 
         curr.name="allClusters.pvalsOrdered", max.size=20)

### Same features, but dont separate by donors

In [ ]:
Idents(se)<-se$seurat_clusters
sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
dot <- DotPlot(se, scale=T,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=F) + RotatedAxis() + ylab("Cluster")
ggsave(file.path(btwnConds_inClust_sepDonrs.outdir,
                 "allClusters.combineDonorsConds.pvalsOrdered.png"))

dot

In [ ]:
all.pvals.nafill <- all.pvals
all.pvals.nafill[,] <- -log10(zoo::na.fill(all.pvals,1))
all.pvals.nafill

Idents(se)<-se$seurat_clusters
sig.all.ordered <- sort(rowMeans(all.pvals.nafill), decreasing=T)
dot <- DotPlot(se, scale=T,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=F) + RotatedAxis() + ylab("Cluster")
ggsave(file.path(btwnConds_inClust_sepDonrs.outdir,
                 "allClusters.combineDonorsConds.pvalsOrderedNA.png"))

dot

In [ ]:
Idents(se)<-se$condition.clust
sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
dot <- DotPlot(se, scale=T,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=F) + RotatedAxis() + ylab("Condition_Cluster")
ggsave(file.path(btwnConds_inClust_sepDonrs.outdir, "allClusters.combineDonors.pvalsOrdered.png"))
dot

In [ ]:
all.pvals.nafill <- all.pvals
all.pvals.nafill[,] <- -log10(zoo::na.fill(all.pvals,1))

Idents(se)<-se$condition.clust
sig.all.ordered <- sort(rowMeans(all.pvals.nafill), decreasing=T)
dot <- DotPlot(se, scale=T,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=F) + RotatedAxis() + ylab("Cluster")
ggsave(file.path(btwnConds_inClust_sepDonrs.outdir,
                 "allClusters.combineDonors.pvalsOrderedNA.png"))

dot

## ii. Taking top 3 of each sig DE

In [ ]:
all.names.sig <- unique(all.names.sig)
de.plots(se, all.names.sig, btwnConds_inClust.outdir, curr.name="allDonors.allClusters", max.size=15)

dot <- DotPlot(se, scale=T,
               features = head(all.names.sig,20), 
               cluster.idents=T) + RotatedAxis()
dot
ggsave(file.path(btwnConds_inClust_sepDonrs.outdir, "allClusters.combineDonors.top3de.png"))
ggsave(file.path(btwnConds_inClust_sepDonrs.outdir, "allClusters.combineDonors.top3de.pdf"))

In [ ]:
# Get commone DE values
# tfs_nclust_set_de_overlap = {} # for each tf count number of clusters seen with de results in at least 1 donor
# tfs_clust_de = {}

# clust_dirs = glob.glob(cond_indir+"/clust_*")
# clusters = [basename(x).split("_")[-1] for x in clust_dirs]
# #clusters.sort()

# for i in clust_dirs:
#     print("clusert", i)
#     clust = basename(i).split("_")[-1]
    
#     for d in range(n_donors):
#         print("donor", d)
#         curr_dir = join(i, f"donor{d}")
#         curr_f = glob.glob(join(curr_dir, "*DE.csv"))
#         assert(len(curr_f)<=1)
#         if len(curr_f) == 0:
#             print("no file")
#             continue
#         curr_f = curr_f[0]
#         curr_de = pd.read_csv(curr_f,index_col=0)
#         #print('curr_de', curr_de.shape)
#         curr_de = curr_de.loc[curr_de["p_val_adj_BH"] < p_thresh]
#         for tf in curr_de.index:
#             if tf in tfs_nclust_set_de_overlap.keys():
#                 tfs_nclust_set_de_overlap[tf].add(clust)
#                     #tfs_nclust_de_overlap[tf] += 1
#                 tfs_clust_de[tf].append((clust, d))
#             else:
#                 #tfs_nclust_de_overlap[tf] = 1
#                 tfs_nclust_set_de_overlap[tf] = set([clust])
#                 tfs_clust_de[tf] = [(clust,d)]
#         print('curr_de sig', curr_de.shape)



--------

## Stimulus specific response in each cluster, not splitting donors

In [ ]:

source(utils_script)

all.pvals <- data.frame(matrix(nrow=dim(se)[1],ncol=0),
                          row.names=row.names(se))
all.names.sig <- c()

#sample_names
cluster.ids <- sort(unique(se$seurat_clusters))
se$celltype.stim <- paste(se$seurat_clusters, se$orig.ident, sep = "_")
se$celltype <- se$seurat_clusters
Idents(se) <- "celltype.stim"

for (curr.clust in cluster.ids){
    ident.1 = paste0(curr.clust, "_", sample_names[[1]]) 
    ident.2 = paste0(curr.clust, "_", sample_names[[2]])
    try({
        print(curr.clust)
        #se.filt <- subset(se, idents = c(ident.1,ident.2))
        #curr.donor = se.filt$donor[[1]] #strsplit(c,"_")[[2]]
        curr.name <-paste0("clust_", curr.clust)
        curr.clust.outdir <- file.path(btwnConds_inClust.outdir, paste0("clust_", curr.clust))
        dir.create(curr.clust.outdir)

        curr.sig <- find.markers.and.plot(se, ident.1, ident.2, curr.clust.outdir, curr.name=curr.clust, 
                              min.pct=minPct, p.thresh=p_thresh,latent.var=latent.vars, test.use=test.use)
        names.sig <- rownames(curr.sig)
        all.names.sig <- c(all.names.sig, head(names.sig, top_de))
        
        all.pvals[names.sig,curr.clust] <- curr.sig$p_val_adj_BH

        curr_clust <- subset(se, seurat_clusters == curr.clust)
        avg_curr_clust <- data.frame(log1p(AverageExpression(curr_clust, verbose = FALSE)$RNA))
        avg_curr_clust$gene <- rownames(avg_curr_clust)
        p1 <- ggplot(avg_curr_clust, aes_string(paste0("X", c, "_", sample_names[[1]]), 
                                                paste0("X", c, "_", sample_names[[2]]))) + geom_point() + ggtitle(paste("Cluster", curr.clust))
        p1 <- LabelPoints(plot = p1, points = names.sig, repel = TRUE)
        plot_grid(p1)
        #write.csv(response, file=file.path(curr.clust.outdir,paste0("cluster_",c,".conditionDE.csv")))
        ggsave(file.path(curr.clust.outdir,paste0("cluster_",curr.clust,".allDonors.conditionScatter.png")))
        
        })
    }
     



## DE sig summarize

### a) top 3 significant results from each comparison

In [ ]:
all.names.sig <- unique(all.names.sig)
de.plots(se, all.names.sig, btwnConds_inClust.outdir, curr.name="allDonors.allClusters", max.size=15)

dot <- DotPlot(se, scale=FALSE,
               features = head(all.names.sig,20), 
               cluster.idents=T) + RotatedAxis()
dot
ggsave(file.path(btwnConds_inClust.outdir, "allDonors.allClusters.top3de.png"))
ggsave(file.path(btwnConds_inClust.outdir, "allDonors.allClusters.top3de.pdf"))

## b) averaged top p-values (only average ones that were DE)

In [ ]:
sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
dot <- DotPlot(se, scale=T,
               features = names(head(sig.all.ordered,20)), 
               cluster.idents=T) + RotatedAxis()
dot
ggsave(file.path(btwnConds_inClust.outdir, "allDonors.allClusters.pvalsOrdered.png"))
ggsave(file.path(btwnConds_inClust.outdir, "allDonors.allClusters.pvalsOrdered.pdf"))
#de.plots(se, names(sig.all.ordered), outdir, curr.name="allDonors.allClusters.pvalsOrdered", max.size=20)

In [ ]:
sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
dot <- DotPlot(se, scale=F,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=T) + RotatedAxis()
dot


--------

In [ ]:
sessionInfo()